In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir(r"C:\Users\Shardul Janaskar\Downloads\domain_classification")

In [3]:
df = pd.read_csv("file.csv")

In [4]:
def func(row):
    for c in df.columns:
        if row[c] =='T':
            return c

df['tag'] = df.apply(func,axis=1)

In [5]:
df.head()

,message,food,recharge,support,reminders,travel,nearby,movies,casual,other,tag
0,7am everyday,F,F,F,T,F,F,F,F,F,reminders
1,chocolate cake,T,F,F,F,F,F,F,F,F,food
2,closed mortice and tenon joint door dimentions,F,F,T,F,F,F,F,F,F,support
3,train eppo kelambum,F,F,F,F,T,F,F,F,F,travel
4,yesterday i have cancelled the flight ticket,F,F,F,F,T,F,F,F,F,travel


In [6]:
df = df[['message','tag']]

In [7]:
sentences = df['message'].tolist()

In [8]:
def preprocess_data(doc_set):
    """
    Input  : docuemnt list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text
    """
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # list for tokenized documents in loop
    texts = []
    # loop through document list
    for i in doc_set:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        # add tokens to list
        texts.append(stemmed_tokens)
    return texts

In [9]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [10]:
processed = preprocess_data(sentences)

In [11]:
token = [" ".join(token) for token in processed]

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
tfidf = TfidfVectorizer()

In [14]:
tfidf.fit(token)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [15]:
X = tfidf.transform(token).toarray()

In [16]:
y = df['tag']

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df['tag'])

In [18]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(X,y,test_size= 0.2,random_state =42)

In [21]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()


In [22]:
nb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:
y_pred = nb.predict(X_test)

In [24]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.79      0.60      0.69      1198
           1       0.85      0.51      0.64       767
           2       0.95      0.61      0.74       437
           3       0.75      0.57      0.65       759
           4       0.89      0.54      0.67       601
           5       0.85      0.57      0.69       497
           6       0.86      0.89      0.88      1446
           7       0.95      0.08      0.15       242
           8       0.57      0.96      0.72      2185

   micro avg       0.72      0.72      0.72      8132
   macro avg       0.83      0.59      0.65      8132
weighted avg       0.77      0.72      0.71      8132

[[ 724    7    2    2    9    6   44    0  404]
 [  16  390    0   29    2   12   27    0  291]
 [  15    3  266   19    3    1    8    0  122]
 [  23   31    7  433    9    2   20    1  233]
 [  46   10    1   23  322    6   23    0  170]
 [  13    6    0    8    3  285   29    0  153]
 [  28 

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7176586325627152

In [26]:
from sklearn.ensemble import RandomForestClassifier


In [27]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]


In [28]:
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [32]:
from sklearn.model_selection import RandomizedSearchCV
rf = RandomForestClassifier()
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [36]:
rf.fit(X_train, y_train)


C:\Users\Shardul Janaskar\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [45]:
ypred=rf.predict(X_test)
yp = rf.predict(X_train)

In [42]:
print(confusion_matrix(y_test,ypred))

[[1048   20    2   14   21   14   37    7   35]
 [ 117  448    2   59    8   35   28    2   68]
 [  44    7  329    9    3    2    8    0   35]
 [  97   44   19  472   32    9   10   19   57]
 [ 100   25    9   38  356   14    7   10   42]
 [  38   35    1    8   11  358   19    1   26]
 [  74   18    6   10    5   15 1291    4   23]
 [  35   17    1   29   10    4   26   87   33]
 [ 190   59    9   46   22   44   55   12 1748]]


In [43]:
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

           0       0.60      0.87      0.71      1198
           1       0.67      0.58      0.62       767
           2       0.87      0.75      0.81       437
           3       0.69      0.62      0.65       759
           4       0.76      0.59      0.67       601
           5       0.72      0.72      0.72       497
           6       0.87      0.89      0.88      1446
           7       0.61      0.36      0.45       242
           8       0.85      0.80      0.82      2185

   micro avg       0.75      0.75      0.75      8132
   macro avg       0.74      0.69      0.70      8132
weighted avg       0.76      0.75      0.75      8132



In [44]:
accuracy_score(y_test,ypred)

0.7546728971962616

In [46]:
accuracy_score(y_train,yp)

0.9628001352722354

In [47]:
import xgboost as xgb

In [48]:
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [50]:
xg = xgb.XGBClassifier(n_estimators = 100)

In [51]:
xg.fit(X_train,y_train)

OSError: [WinError -529697949] Windows Error 0xe06d7363